In [18]:
#Following tutorial https://www.youtube.com/watch?v=vQhEiR2bNY8

#Set up the client

from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY")
)

In [19]:
# Declare Assistant ID

assistant_id = "asst_wmUPInPNiI5UnWNb001ehA5r"

In [20]:
# Declare Functions

def add(num1, num2):
    return num1+num2

def subtract(num1, num2):
    return num1-num2


5
3


In [21]:
assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)

assistant

Assistant(id='asst_wmUPInPNiI5UnWNb001ehA5r', created_at=1707805935, description=None, file_ids=[], instructions='You are a helpful math tutor that helps students with simple mathematics, such as adding and subtracting.', metadata={}, model='gpt-3.5-turbo', name='Math tutor', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract from.'}, 'num_2': {'type': 'integer', 'description': 'The number to subtract by.'}}, 'required': ['num_1', 'num_2']}), type='function')])

In [52]:
# Create a thread

thread = client.beta.threads.create()
thread

Thread(id='thread_lh183M12yrlseMrGcCquv7vF', created_at=1707809894, metadata={}, object='thread')

In [53]:
# Prompt the model to add something

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Do (15+22)-2"
)
run

Run(id='run_h9RYwzwtqMDxPDrjt9rNto9C', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707809908, expires_at=1707810508, failed_at=None, file_ids=[], instructions='Do (15+22)-2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_lh183M12yrlseMrGcCquv7vF', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract from.'}

In [56]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

#run.completed_at is possible

Run(id='run_h9RYwzwtqMDxPDrjt9rNto9C', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707809908, expires_at=1707810508, failed_at=None, file_ids=[], instructions='Do (15+22)-2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_AhX3s1ymAMJD8SuwJYDaFScG', function=Function(arguments='{\n  "num_1": 15,\n  "num_2": 22\n}', name='add'), type='function')]), type='submit_tool_outputs'), started_at=1707809908, status='requires_action', thread_id='thread_lh183M12yrlseMrGcCquv7vF', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'requi

In [60]:
import json

def get_outputs_for_tool_call(tool_call):
    num1 = json.loads(tool_call.function.arguments)["num_1"]
    num2 = json.loads(tool_call.function.arguments)["num_2"]

    print(tool_call.function)

    if tool_call.function.name == 'add':
        desired = add(num1, num2)
    elif tool_call.function.name == 'subtract':
        desired = subtract(num1, num2)

    return {
        "tool_call_id": tool_call.id,
        "output": desired
    }

tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(get_outputs_for_tool_call, tool_calls)
tool_outputs = list(tool_outputs)
tool_outputs

    

Function(arguments='{\n  "num_1": 37,\n  "num_2": 2\n}', name='subtract')


[{'tool_call_id': 'call_TFjjvx0IBxSnjEtHjCph8B8j', 'output': 35}]

In [61]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

run

Run(id='run_h9RYwzwtqMDxPDrjt9rNto9C', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=None, created_at=1707809908, expires_at=1707810508, failed_at=None, file_ids=[], instructions='Do (15+22)-2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1707809947, status='queued', thread_id='thread_lh183M12yrlseMrGcCquv7vF', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtract f

In [62]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

run

Run(id='run_h9RYwzwtqMDxPDrjt9rNto9C', assistant_id='asst_wmUPInPNiI5UnWNb001ehA5r', cancelled_at=None, completed_at=1707809971, created_at=1707809908, expires_at=None, failed_at=None, file_ids=[], instructions='Do (15+22)-2', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1707809970, status='completed', thread_id='thread_lh183M12yrlseMrGcCquv7vF', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add', description='Adds two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The first number to add.'}, 'num_2': {'type': 'integer', 'description': 'The second number to add.'}}, 'required': ['num_1', 'num_2']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='subtract', description='Subtracts two numbers together', parameters={'type': 'object', 'properties': {'num_1': {'type': 'integer', 'description': 'The number to subtrac

In [63]:
# Show the latest message.

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
messages.data[0].content[0].text.value

'The result of (15+22)-2 is 35.'